In [13]:
import pandas as pd
import numpy as np
#import pylab as pl
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

from util.methods import Methods
from util.receiving import Receiving
from util.stat import FantasyStats

from cost_functions import CostFunction
pd.set_option('display.width', 300)
# Update this to the latest year that data is available
score_year = 2016

In [14]:
func = Methods()
receiving = Receiving(2002, score_year)
receiving.compile_data()
receiving.construct_feature_vector()
df=receiving.get_receiving_data()
df[receiving.dep_var + receiving.feature_vector].describe()

,y_future_rank,y_top5,y_sqrt_future_rank,y_future_points,y_sqr_future_points,RK,is_rec,games_played,n_years,td_per_game,...,percent_comp,avg_career_rank,previous_rank1_clean,previous_rank2_clean,previous_rank3_clean,previous_rank4_clean,previous_rank5_clean,lst_rank_delta,r_td_rec,log_yards
count,458.000000,722.000000,458.000000,458.000000,458.000000,722.000000,722.000000,722.000000,722.000000,722.000000,...,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000
mean,27.327511,0.134349,4.895174,177.860262,13.221885,30.554017,0.808864,14.797784,3.673130,0.415197,...,0.645613,30.496999,29.781163,30.059557,30.641274,30.815789,31.130194,0.633783,0.087705,6.824655
std,17.250432,0.341263,1.836341,47.683931,1.746043,17.367705,0.393468,1.415427,2.635209,0.216680,...,0.128298,14.377992,17.264099,17.204185,16.949468,16.839970,16.774996,2.747029,0.042968,0.238354
min,1.000000,0.000000,1.000000,92.000000,9.591663,1.000000,0.000000,7.000000,1.000000,0.000000,...,0.393333,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-0.977273,0.000000,6.342121
25%,NaN,0.000000,NaN,NaN,NaN,16.000000,1.000000,15.000000,1.000000,0.250000,...,0.564915,19.666667,15.000000,15.000000,17.000000,17.000000,17.000000,-0.217664,0.056604,6.632331
50%,NaN,0.000000,NaN,NaN,NaN,31.000000,1.000000,15.000000,3.000000,0.384615,...,0.620586,29.250000,30.000000,30.000000,30.000000,30.000000,31.000000,0.000000,0.082192,6.790659
75%,NaN,0.000000,NaN,NaN,NaN,46.000000,1.000000,16.000000,5.000000,0.533333,...,0.681248,42.000000,44.000000,44.000000,44.000000,44.000000,44.000000,0.432292,0.112676,6.989796
max,60.000000,1.000000,7.745967,347.000000,18.627936,60.000000,1.000000,16.000000,13.000000,1.437500,...,1.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,39.000000,0.250000,7.582738


In [15]:
df.to_csv('/Users/mikhail/combust/combust.ml-demo/data/fantasy_football/receiving.csv', index=False)

In [16]:
stat = FantasyStats(df, score_year)
stat.build_train_valid_score(60)
df_train = stat.get_df_train()
print len(df_train)

df_valid = stat.get_df_valid()
print len(df_valid)

df_score = stat.get_df_score()
print len(df_score)

400
262
60


### Train the Gradient Boosted Regression

In [17]:
params = {'n_estimators': 1000,
          'max_depth': 3,
          'min_samples_split': 2,
          'learning_rate': 0.04,
          'loss': 'ls',
          'verbose': 1}

dependent_variable = receiving.dep_var[2]
print dependent_variable

stat.train_gbr_fulL_sample(dependent_variable, params, receiving.feature_vector)

clfr = stat.get_gbr_result()
print "\n"
stat.feature_importance(clfr, receiving.feature_vector)

y_sqrt_future_rank
      Iter       Train Loss   Remaining Time 
         1           3.2979            0.94s
         2           3.2369            0.85s
         3           3.1801            0.78s
         4           3.1268            0.76s
         5           3.0738            0.79s
         6           3.0258            0.77s
         7           2.9793            0.76s
         8           2.9373            0.74s
         9           2.8990            0.74s
        10           2.8625            0.76s
        20           2.5449            0.75s
        30           2.3080            0.72s
        40           2.1381            0.70s
        50           2.0134            0.67s
        60           1.9139            0.64s
        70           1.8249            0.62s
        80           1.7552            0.61s
        90           1.6913            0.61s
       100           1.6323            0.59s
       200           1.0646            0.50s
       300           0.7102        

### Train the Sqrt Points Gradient Boosted Regression

In [18]:
params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 2,
          'learning_rate': 0.04,
          'loss': 'ls',
          'verbose': 1}

dependent_variable = receiving.dep_var[4]
print dependent_variable

stat.train_gbr_fulL_sample(dependent_variable, params, receiving.feature_vector)

clfr2 = stat.get_gbr_result()
print "\n"
stat.feature_importance(clfr2, receiving.feature_vector)

y_sqr_future_points
      Iter       Train Loss   Remaining Time 
         1           2.9764            0.40s
         2           2.9159            0.41s
         3           2.8599            0.39s
         4           2.8081            0.38s
         5           2.7595            0.37s
         6           2.7117            0.37s
         7           2.6681            0.36s
         8           2.6246            0.36s
         9           2.5816            0.36s
        10           2.5437            0.38s
        20           2.2305            0.38s
        30           2.0293            0.37s
        40           1.8881            0.36s
        50           1.7703            0.33s
        60           1.6792            0.33s
        70           1.6092            0.32s
        80           1.5377            0.30s
        90           1.4895            0.29s
       100           1.4365            0.28s
       200           0.9859            0.20s
       300           0.6826       

### Train the Gradient Boosted Classifier

In [19]:
params = {'n_estimators': 400, 
          'learning_rate': 0.04,
          'random_state': 0,
          'max_depth': 3,
          'min_samples_split': 2,
          'verbose': 1, 
          'random_state': 0}
 

dependent_variable = receiving.dep_var[1]
print dependent_variable
stat.train_gbc_full_sample(dependent_variable, params, receiving.feature_vector)
print "\n"
clfc = stat.get_gbc_result()
stat.feature_importance(clfc, receiving.feature_vector)

y_top5
      Iter       Train Loss   Remaining Time 
         1           0.7672            0.53s
         2           0.7496            0.50s
         3           0.7334            0.55s
         4           0.7195            0.54s
         5           0.7066            0.55s
         6           0.6957            0.55s
         7           0.6860            0.54s
         8           0.6763            0.53s
         9           0.6675            0.53s
        10           0.6594            0.52s
        20           0.5980            0.51s
        30           0.5571            0.46s
        40           0.5243            0.43s
        50           0.5003            0.42s
        60           0.4786            0.40s
        70           0.4570            0.40s
        80           0.4349            0.38s
        90           0.4126            0.37s
       100           0.3962            0.36s
       200           0.2590            0.23s
       300           0.1831            0.12s
  

### Validation sample (boosted reg 1)

In [20]:
df_combined = stat.score_train_and_validation_reg(clfr, receiving.feature_vector, True)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

/Users/mikhail/combust/venv/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,PLAYER,TEAM,y_future_rank,p_rank
464,"T.Y. Hilton, WR",IND,1.0,1.0
331,"Julio Jones, WR",ATL,2.0,2.0
588,"Odell Beckham Jr., WR",NYG,3.0,3.0
411,"Antonio Brown, WR",PIT,5.0,4.0
239,"Mike Evans, WR",TB,4.0,5.0
307,"Amari Cooper, WR",OAK,8.0,6.0
288,"Brandin Cooks, WR",NO,7.0,7.0
8,"Jarvis Landry, WR",MIA,10.0,8.0
336,"Doug Baldwin, WR",SEA,11.0,9.0
22,"Calvin Johnson, WR",DET,NaN,10.0


### Validation sample (boosted reg 2)

In [21]:
df_combined = stat.score_train_and_validation_reg(clfr2, receiving.feature_vector, False)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

/Users/mikhail/combust/venv/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,PLAYER,TEAM,y_future_rank,p_rank
588,"Odell Beckham Jr., WR",NYG,3.0,1.0
411,"Antonio Brown, WR",PIT,5.0,2.0
239,"Mike Evans, WR",TB,4.0,3.0
331,"Julio Jones, WR",ATL,2.0,4.0
652,"Eric Decker, WR",NYJ,NaN,5.0
464,"T.Y. Hilton, WR",IND,1.0,6.0
198,"Rob Gronkowski, TE",NE,NaN,7.0
497,"Sammy Watkins, WR",BUF,NaN,8.0
80,"Martavis Bryant, WR",PIT,NaN,9.0
135,"James Jones, WR",GB,NaN,10.0


### Validation sample (boosted classifier 1)

In [22]:
df_combined = stat.score_train_and_validation_class(clfc, receiving.feature_vector, False)
df_combined[df_combined.year==score_year-1].sort('p_rank', ascending=True)[['PLAYER', 'TEAM', 'y_future_rank', 'p_rank']][:15]

/Users/mikhail/combust/venv/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,PLAYER,TEAM,y_future_rank,p_rank
331,"Julio Jones, WR",ATL,2.0,1.0
411,"Antonio Brown, WR",PIT,5.0,2.0
588,"Odell Beckham Jr., WR",NYG,3.0,3.0
239,"Mike Evans, WR",TB,4.0,4.0
8,"Jarvis Landry, WR",MIA,10.0,5.0
464,"T.Y. Hilton, WR",IND,1.0,6.0
307,"Amari Cooper, WR",OAK,8.0,7.0
288,"Brandin Cooks, WR",NO,7.0,8.0
317,"Emmanuel Sanders, WR",DEN,19.0,9.0
671,"Randall Cobb, WR",GB,NaN,10.0


In [23]:
def _final_order_score(df):
    score = float(df['pred_boost_reg1']) * 0.4 +\
            float(df['pred_boost_reg2']) * 0.4 +\
            float(df['pred_classifier1']) * 0.2
    return score

In [24]:
df_score_reg = pd.DataFrame(stat.score_pred_reg(clfr, receiving.feature_vector, True))
df_score_reg.set_index('PLAYER', inplace=True, drop=False)
df_score_reg.rename(columns={'p_rank': 'pred_boost_reg1'}, inplace=True)

df_score_reg2 = pd.DataFrame(stat.score_pred_reg(clfr2, receiving.feature_vector, False))
df_score_reg2.set_index('PLAYER', inplace=True, drop=False)
df_score_reg2.rename(columns={'p_rank': 'pred_boost_reg2'}, inplace=True)


df_score_class = pd.DataFrame(stat.score_pred_class(clfc, receiving.feature_vector, False))
df_score_class.set_index('PLAYER', inplace=True, drop=False)
df_score_class.rename(columns={'p_rank': 'pred_classifier1'}, inplace=True)

df_score_all = df_score_reg.join(df_score_class['pred_classifier1'])
df_score_all = df_score_all.join(df_score_reg2['pred_boost_reg2'])

df_score_all['final_order_score'] = df_score_all.apply(_final_order_score, axis=1)
df_score_all.sort('final_order_score', ascending=True, inplace=True)
df_score_all.reset_index(inplace=True, drop=True)
df_score_all.rename(columns={'RK': '%s Rank' % score_year}, inplace=True)

df_score_all[['PLAYER', 'TEAM', '%s Rank' % score_year, 'final_order_score', 'pred_boost_reg1', 'pred_boost_reg2', 'pred_classifier1']]

/Users/mikhail/combust/venv/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,PLAYER,TEAM,2016 Rank,final_order_score,pred_boost_reg1,pred_boost_reg2,pred_classifier1
0,"Michael Thomas, WR",NO,9.0,1.2,1.0,1.0,2.0
1,"Tyrell Williams, WR",LAC,17.0,2.6,4.0,2.0,1.0
2,"Odell Beckham Jr., WR",NYG,3.0,3.6,3.0,4.0,4.0
3,"Julio Jones, WR",ATL,2.0,5.0,6.0,5.0,3.0
4,"Amari Cooper, WR",OAK,8.0,6.6,5.0,8.0,7.0
5,"Jordy Nelson, WR",GB,6.0,10.0,10.0,3.0,24.0
6,"Michael Crabtree, WR",OAK,24.0,10.6,7.0,7.0,25.0
7,"Jarvis Landry, WR",MIA,10.0,11.6,9.0,14.0,12.0
8,"Allen Robinson, WR",JAX,37.0,11.6,8.0,18.0,6.0
9,"Antonio Brown, WR",PIT,5.0,13.4,13.0,10.0,21.0


In [25]:
df_score_all[['PLAYER', 'TEAM', '%s Rank' % score_year, 'final_order_score', 'pred_boost_reg1', 'pred_boost_reg2', 'pred_classifier1']].to_csv('/Users/mikhail/Documents/nfl_receiving_2017.tsv', sep='\t')